In [ ]:
import sys

## Comment this out if you don't want to install pennylane from this notebook
!{sys.executable} -m pip install pennylane

## Comment this out if you don't want to install matplotlib from this notebook
!{sys.executable} -m pip install matplotlib

# Differentiable quantum computing with PennyLane

In this tutorial we will:

* learn step-by-step how quantum computations are implemented in PennyLane,
* understand parameter-dependent quantum computations ("variational circuits"), 
* build our first quantum machine learning model, and
* compute its gradient.

In [ ]:
import pennylane as qml
from pennylane import numpy as np

### Quantum devices

In PennyLane, a *quantum node* is a computational unit that involves the construction, evaluation, pre- and postprocessing of quantum computations.

A quantum node consists of a *quantum function* that defines a circuit, as well as a *device* on which it is run. 

There is a growing [device ecosystem](https://pennylane.ai/plugins.html) which allows you to change only one line of code to dispatch your quantum computation to local simulators, remote simulators and remote hardware from different vendors.

Here we will use the in-built `default.qubit` device.

In [ ]:
dev = qml.device('default.qubit', wires=2)

### The initial state

<br />
<img src="figures/1.png" width="550" height="400">
<br />

The initial state has 100% probability to be measured in the "0..0" configuration. Let's see how we can verify this with PennyLane.

In [ ]:
@qml.qnode(dev)
def circuit():
    return qml.probs(wires=[0, 1])

circuit()

tensor([1., 0., 0., 0.], requires_grad=True)

The internal state vector that we use to mathematically keep track of probabilities is complex-valued. Since `default.qubit` is a simulator we can have a look at the state, for example by checking the device's `state` attribute.

In [ ]:
dev.state

tensor([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j], requires_grad=True)

### Unitary evolutions

<br />
<img src="figures/2.png" width="650" height="400">
<br />

Quantum circuits are represented by unitary matrices. We can evolve the initial state by an arbitrary unitrary matrix as follows:

In [ ]:
s = 1/np.sqrt(2)
U = np.array([[0., -s, 0.,  s],
              [ s, 0., -s, 0.],
              [ s, 0.,  s, 0.],
              [0., -s, 0., -s]])

@qml.qnode(dev)
def circuit():
    qml.QubitUnitary(U, wires=[0, 1])
    return qml.probs(wires=[0, 1])

circuit()

tensor([0. , 0.5, 0.5, 0. ], requires_grad=True)

The internal quantum state changed.

In [ ]:
dev.state

tensor([0.        +0.j, 0.70710678+0.j, 0.70710678+0.j, 0.        +0.j], requires_grad=True)

### Measurements sample outcomes from the distribution

<br />
<img src="figures/3.png" width="650" height="400">
<br />

The most common measurement takes samples $-1, 1$ from the "Pauli-Z" observable. The samples indicate if the qubit was measured in state $| 0 \rangle$ or $| 1 \rangle$.

In [ ]:
@qml.qnode(dev)
def circuit():
    qml.QubitUnitary(U, wires=[0, 1])
    return qml.sample(qml.PauliZ(wires=0)), qml.sample(qml.PauliZ(wires=1))

circuit()

tensor([[-1,  1, -1, ...,  1,  1, -1],
        [ 1, -1,  1, ..., -1, -1,  1]], requires_grad=True)

The quantum state should be still the same as above.

In [ ]:
dev.state

tensor([0.        +0.j, 0.70710678+0.j, 0.70710678+0.j, 0.        +0.j], requires_grad=True)

### Computing expectation values 

<br />
<img src="figures/4.png" width="650" height="400">
<br />

When we want outputs of computations to be deterministic, we often interpret the expected measurement outcome as the result. This value is estimated by taking lots of samples and averaging over them.

In [ ]:
@qml.qnode(dev)
def circuit():
    qml.QubitUnitary(U, wires=[0, 1])
    return qml.expval(qml.PauliZ(wires=0)), qml.expval(qml.PauliZ(wires=1))

circuit()

tensor([0., 0.], requires_grad=True)

Again, the quantum state should be still the same as above.

In [ ]:
dev.state

tensor([0.        +0.j, 0.70710678+0.j, 0.70710678+0.j, 0.        +0.j], requires_grad=True)

### Quantum circuits are decomposed into gates

<br />
<img src="figures/5.png" width="650" height="400">
<br />

Quantum circuits rarely consist of one large unitary (which quickly becomes intractably large as the number of qubits grow). Instead, they are composed of *quantum gates*.

In [ ]:
@qml.qnode(dev)
def circuit():
    qml.PauliX(wires=0)
    qml.CNOT(wires=[0,1])
    qml.Hadamard(wires=0)
    qml.PauliZ(wires=1)
    return qml.expval(qml.PauliZ(wires=0)), qml.expval(qml.PauliZ(wires=1))

circuit()

tensor([ 0., -1.], requires_grad=True)

### Some gates depend on "control" parameters

<br />
<img src="figures/6.png" width="650" height="400">
<br />

To train circuits, there is a special subset of gates which is of particular interest: the Pauli rotation gates. These "rotate" a special representation of the quantum state around a specific axis. The gates depend on a scalar parameter which is the angle of the rotation. 

In [ ]:
@qml.qnode(dev)
def circuit(w1, w2):
    qml.RX(w1, wires=0)
    qml.CNOT(wires=[0,1])
    qml.RY(w2, wires=1)
    return qml.expval(qml.PauliZ(wires=0)), qml.expval(qml.PauliZ(wires=1))

circuit(0.2, 1.3)

The names `w1`, `w2` are already suggestive that these can be used like the trainable parameters of a classical machine learning model. But we could also call the control parameters `x1`, `x2` and encode data features into quantum states. 

### A full quantum machine learning model

Finally, we can use pre-coded routines or [templates](https://pennylane.readthedocs.io/en/stable/introduction/templates.html) to conveniently build full quantum machine learning model that include a data encoding part, and a trainable part.

<br />
<img src="figures/7.png" width="650" height="400">
<br />

Here, we will use the `AngleEmbedding` template to load the data, and the `BasicEntanglingLayers` as the trainable part of the circuit.

In [ ]:
@qml.qnode(dev)
def quantum_model(x, w):
    qml.templates.AngleEmbedding(x, wires=[0, 1])
    qml.templates.BasicEntanglerLayers(w, wires=[0, 1])
    return qml.expval(qml.PauliZ(wires=0))


x = np.array([0.1, 0.2], requires_grad=False)
w = np.array([[-2.1, 1.2], [-1.4, -3.9], [0.5, 0.2]])

quantum_model(x, w)

tensor(0.012023, requires_grad=True)

We can draw the circuit.

In [ ]:
print(quantum_model.draw())

 0: ──RX(0.1)──RX(-2.1)──╭C──RX(-1.4)──╭C──RX(0.5)──╭C──┤ ⟨Z⟩ 
 1: ──RX(0.2)──RX(1.2)───╰X──RX(-3.9)──╰X──RX(0.2)──╰X──┤     



The best thing is that by using PennyLane, we can easily compute its gradient!

In [ ]:
gradient_fn = qml.grad(quantum_model)

gradient_fn(x, w)

This allows us to slot the quantum circuit into the machine learning example from the previous notebook.

#  ====== TASKS =============

1. Copy and paste the code from the previous notebook to here and replace the classical model by 
   the `quantum_model` function. This will allow you to train the model!

2. Add a bias term to the quantum model.

3. Replace the hand-coded optimisation step by a native [PennyLane optimiser](https://pennylane.readthedocs.io/en/stable/introduction/optimizers.html).

4. Rewrite the entire example in PyTorch. 

   Tipp: You must set the qnode to the correct interface via `@qml.qnode(dev, interface='tf')`.